In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageOps
import numpy as np
import glob

In [2]:
# 이미지 리사이징 및 검정색 배경으로 채우는 함수
def resize_image_fill_black(image_path, target_size):
    image = Image.open(image_path)
    new_image = ImageOps.fit(image, target_size, method=0, bleed=0.0, centering=(0.5, 0.5), fillcolor=(0, 0, 0))
    resized_image = np.array(new_image)
    return resized_image

In [3]:
# 데이터셋 경로 설정
train_directory = 'D:/note_deep_learning/train_note'
test_directory = 'D:/note_deep_learning/test_note'

# 데이터 증강 및 리사이징 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 및 테스트 데이터셋 로드
target_size = (64, 64)  # 원하는 크기로 변경
batch_size = 32

In [4]:
training_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 9000 images belonging to 9 classes.


In [5]:
test_set = test_datagen.flow_from_directory(
    test_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 965 images belonging to 9 classes.


In [6]:
print("Training Set 클래스 이름: ", training_set.class_indices)
print("Test Set 클래스 이름: ", test_set.class_indices)

Training Set 클래스 이름:  {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8}
Test Set 클래스 이름:  {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8}


In [7]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(9, activation='softmax'))
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)#epochs=훈련횟수, 결과 값 안나오면 숫자 늘려가기


Epoch 1/25
282/282 [==============================] - 31s 108ms/step - loss: 0.9823 - accuracy: 0.6693 - val_loss: 0.3710 - val_accuracy: 0.8746
Epoch 2/25
282/282 [==============================] - 33s 117ms/step - loss: 0.4267 - accuracy: 0.8518 - val_loss: 0.2306 - val_accuracy: 0.9347
Epoch 3/25
282/282 [==============================] - 36s 128ms/step - loss: 0.2840 - accuracy: 0.9062 - val_loss: 0.2006 - val_accuracy: 0.9409
Epoch 4/25
282/282 [==============================] - 33s 116ms/step - loss: 0.2074 - accuracy: 0.9306 - val_loss: 0.1788 - val_accuracy: 0.9440
Epoch 5/25
282/282 [==============================] - 33s 117ms/step - loss: 0.1677 - accuracy: 0.9433 - val_loss: 0.1876 - val_accuracy: 0.9409
Epoch 6/25
282/282 [==============================] - 34s 121ms/step - loss: 0.1353 - accuracy: 0.9572 - val_loss: 0.1304 - val_accuracy: 0.9648
Epoch 7/25
282/282 [==============================] - 31s 108ms/step - loss: 0.1182 - accuracy: 0.9601 - val_loss: 0.1217 - val_ac

In [ ]:
from tensorflow.keras.preprocessing import image

# 이미지 파일이 있는 디렉토리 경로
image_directory = 'D:/note_deep_learning/test/'

# 이미지 파일 목록 가져오기
image_files = glob.glob(image_directory + '*.jpg')  # 필요한 이미지 확장자에 따라 수정

# 클래스 인덱스와 클래스 이름 매핑
class_indices = training_set.class_indices
class_names = {v: k for k, v in class_indices.items()}

# 각 이미지에 대한 예측 수행
for image_file in image_files:
    # 이미지 로드 및 전처리
    test_image = image.load_img(image_file, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    # 예측 수행
    result = cnn.predict(test_image)

    # 예측 결과 확인
    predicted_class_index = np.argmax(result)
    predicted_class_name = class_names[predicted_class_index]

    print(f"Image: {image_file}, Predicted Class: {predicted_class_name}")